In [1]:
import numpy as np

In [27]:
class Layer:
    """
    Base class for layers in the network.
    Arguments:
        `inbound_layers`: A list of layers with edges into this layer.
    """
    def __init__(self, input_layer=None):
        """ Simple constructor """
        self.input_layer = input_layer
        self.outbound_layers = []
        
        if self.input_layer:
            self.input_layer.outbound_layers.append(self)

    def forward():
        """
        Every layer that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward():
        """
        Every layer that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError


In [28]:
"""Add your hidden layer here """
class Input(Layer):
    """ Represets all the Hidden Layers """
    def __init__(self, weights, bias):
        Layer.__init__(self)
        self.X = None
        self.W = weights
        self.biases = bias

        self.num_neurons = self.W.shape[-1]
        self.num_images = self.W.shape[0]
        self.num_features = self.W.shape[1]

        self.W_gradients = np.empty((self.num_images, self.num_features))
        self.b_gradients = np.empty((self.num_images,))
        self.gradients = np.empty((self.num_images, self.num_features))

        self.value = np.empty((self.num_images, self.num_neurons))

    def _set(self, images):
        self.X = images
        
    def forward(self):
        """Layer: Input.  execute W * X + b for the entire batch"""
        for i in range(self.num_images):
            h = np.dot(self.X[i], self.W[i])
            self.value[i] = h + self.biases[i]

    def backward(self):
        """ Gradient: Input Layer"""
        self.input_gradients = self.outbound_layers[0].gradients
        for i in range(self.num_images):
            self.W_gradients[i] = np.dot(self.W[i], self.input_gradients[i])
            self.b_gradients[i] = np.dot(self.input_gradients[i], self.biases[i])

            self.gradients[i] = self.W_gradients[i] * self.b_gradients[i]


In [29]:
class Softmax(Layer):

    
    def __init__(self, input_layer):
        Layer.__init__(self, input_layer)

        self.num_classes = self.input_layer.num_classes
        self.num_images = self.input_layer.num_images

        self.value = np.empty((self.num_images, self.num_classes))
        self.gradients = np.empty((self.num_images, self.num_classes))

    def _softmax(self, x):
        """
        Calculate Sigmoid
        `x`: A numpy array-like object.
        """
        return np.exp(x) / np.sum(np.exp(x), axis=0)

    def forward(self):
        """ Perform the sigmoid function and set the value. """
        input_value = self.input_layer.value
        for i in range(self.num_images):
            self.value[i] = self._softmax(input_value[i])


    def backward(self):
        """
        Calculates the gradient using the derivative of
 the softmax function.
        """
        for i in range(self.num_images):
            self.gradients[i] = self.value[i].T